In [10]:
import pandas as pd
from pytrends.request import TrendReq
import numpy as np

In [11]:

def tends(keyw):
    user_timeframe=fechaIni+' '+fechaFin
    user_cat=7
    #'Finance'
    pytrend = TrendReq(hl='es-MX', tz=360)
    
    pytrend.build_payload(
         kw_list=keyw,
         cat=0,
         timeframe=user_timeframe,
         geo='MX',
         gprop='')
    data = pytrend.interest_over_time()
    data= data.drop(labels=['isPartial'],axis='columns').reset_index()
    
    return data


In [12]:
def maxTendencia(wordsTot):
    keywords = wordsTot

    listWord=[]
    for i in keywords:
        listWord.append(i)
        print(listWord)
        data0 =tends(listWord)
        df0 =pd.DataFrame(data0[listWord].max()==100,columns=['bool']).reset_index()
        varMax=list(df0['index'][df0['bool']==True])

        if i==varMax[0]: listWord=varMax
        else: listWord.remove(i)

    print(listWord)
    return listWord, data0


In [13]:
def obtenerTendencia(listWord, keywords):
    key0 = listWord
    #dataf = data0[['date']]
    dataf = pd.DataFrame()
    for i in keywords:
        if i!=key0[0]: 
            key0.append(i)
            print(key0)
            data1 = tends(key0)

            if not dataf.empty:
                listVarSel=list(set(data1.columns) - set(dataf['variable'].unique()))
                data1 = data1[listVarSel]

            colmelt=list(set(data1.columns)-{'date'})
            dataf = dataf.append(data1.melt(id_vars='date', value_vars=colmelt, value_name='valor'))

            #dataf = dataf.drop_duplicates()
            #data1 = data1[[i]]
            #dataf = pd.concat([dataf,data1],axis=1)
            key0.remove(i)

    return dataf

In [14]:

def banc(x):
    ba=""
    if 'santander' in x: ba='SANTANDER PR'
    elif 'bbva' in x or 'bancomer' in x: ba='BANCOMER'
    elif 'banorte' in x: ba='BANORTE'
    elif 'hsbc' in x: ba='HSBC PR'
    elif 'banamex' in x or 'citi' in x: ba='BANAMEX'
    else: ba='OTRO'

    return ba

def codBase(dataf, etiqueta):

    #colsDataf = list(set(dataf.columns)-{'date'})
    #dataf2 = dataf.melt(id_vars='date',value_vars=colsDataf,value_name='valor')

    #dataf2.loc[dataf2['variable'].str.contains('santander'),'banco']='SANTANDER PR'

    dataf['banco'] = dataf['variable'].map(banc)

    
    dataf3 = dataf.groupby(['banco','date']).agg(sm=('valor','sum')).reset_index()
    dataf3['fh_corte'] = pd.to_datetime(dataf3['date'].dt.strftime('%Y-%m-'+'01'),format='%Y-%m-%d')
    
    dataf4 = dataf3.groupby(['banco','fh_corte']).agg(mn=('sm','mean')).reset_index()
    dataf4['tendencia'] = dataf4['mn']/dataf4['mn'].max()
    #dataf4['tendencia'] = dataf4['mn']/dataf4.groupby('fh_corte').mn.transform(np.sum)
    
    dataf4['etiqueta'] = etiqueta
    
    return dataf4


In [15]:
# Consumo

bancos = ['bancomer','bbva','bbva bancomer','santander','banamex','citibanamex', 'citi', 'banorte','hsbc',
          'scotiabank','banco azteca','bancoppel'
         ]

words=['prestamo en bancomer','prestamo bancomer', 'prestamos bancomer','bancomer prestamo nomina','prestamo personal bancomer', 
       'bancomer prestamo de nomina', 'bancomer prestamo personal', 'solicitar prestamo bancomer', 'bancomer prestamo']
wordsTotConsumo = []
for i in bancos:
    for j in words:
        wordsTotConsumo.append(j.replace('bancomer',i))
        
        
#Tarjetas tdc

words=['bancomer solicitar tarjeta de credito','bancomer tarjeta de credito','solicitar tarjeta de credito bancomer','solicitud tarjeta de credito bancomer',
'tarjeta bancomer','tarjeta de credito bancomer','tarjeta de crédito bancomer','tarjeta de credito bancomer sin anualidad','tarjetas bancomer',
'tarjetas de credito bancomer','tramitar tarjeta de credito bancomer','bancomer tarjeta de crédito','tdc bancomer','bancomer tdc']
wordsTotTdc = []
for i in bancos:
    for j in words:
        wordsTotTdc.append(j.replace('bancomer',i))       

#nómina

words=['bancomer nomina','bancomer nómina','bancomer portabilidad de nomina','portabilidad de nómina bancomer','portabilidad de nomina bancomer','bancomer portabilidad de nómina','bancomer portabilidad nomina','bancomer portabilidad nómina',
      'bancomer cambio de nómina','cambio de nómina bancomer','cambio nómina bancomer']
wordsTotNom = []
for i in bancos:
    for j in words:
        wordsTotNom.append(j.replace('bancomer',i)) 
    
#vista

words=['cuenta bancomer','cuenta de ahorro bancomer','bancomer cuenta de ahorro','cuenta ahorro bancomer',
       'cuenta débito bancomer','cuenta debito bancomer','bancomer cuenta debito','cuenta de debito bancomer',
      'bancomer cuenta depósito','bancomer cuenta deposito','cuenta deposito bancomer','cuenta depósito bancomer','cuenta de deposito bancomer',
       'cuenta de depósito bancomer']
wordsTotVist = []
for i in bancos:
    for j in words:
        wordsTotVist.append(j.replace('bancomer',i))     
    
    

In [16]:
ruta='/Users/mb66483/Documents/'
fechaIni='2017-01-01'
fechaFin='2020-10-01'
wordsTot = 'wordsTotVist'
#wordsTotConsumo #wordsTotTdc #wordsTotNom #wordsTotVist

if wordsTot=='wordsTotConsumo': 
    wordVec = wordsTotConsumo
    etiqueta='goog_tend_consumo'
elif wordsTot=='wordsTotTdc': 
    wordVec = wordsTotTdc
    etiqueta='goog_tend_tdc'
elif wordsTot=='wordsTotNom': 
    wordVec = wordsTotNom
    etiqueta='goog_tend_nom'
elif wordsTot=='wordsTotVist': 
    wordVec = wordsTotVist
    etiqueta='goog_tend_vista'
    
listWord, data0= maxTendencia(wordVec)
datafl = obtenerTendencia(listWord,wordVec)
data4 = codBase(datafl,etiqueta)

data4.to_excel(ruta+'exportls.xlsx')

['cuenta bancomer']
['cuenta bancomer', 'cuenta de ahorro bancomer']
['cuenta bancomer', 'bancomer cuenta de ahorro']
['cuenta bancomer', 'cuenta ahorro bancomer']
['cuenta bancomer', 'cuenta débito bancomer']
['cuenta bancomer', 'cuenta debito bancomer']
['cuenta bancomer', 'bancomer cuenta debito']
['cuenta bancomer', 'cuenta de debito bancomer']
['cuenta bancomer', 'bancomer cuenta depósito']
['cuenta bancomer', 'bancomer cuenta deposito']
['cuenta bancomer', 'cuenta deposito bancomer']
['cuenta bancomer', 'cuenta depósito bancomer']
['cuenta bancomer', 'cuenta de deposito bancomer']
['cuenta bancomer', 'cuenta de depósito bancomer']
['cuenta bancomer', 'cuenta bbva']
['cuenta bancomer', 'cuenta de ahorro bbva']
['cuenta bancomer', 'bbva cuenta de ahorro']
['cuenta bancomer', 'cuenta ahorro bbva']
['cuenta bancomer', 'cuenta débito bbva']
['cuenta bancomer', 'cuenta debito bbva']
['cuenta bancomer', 'bbva cuenta debito']
['cuenta bancomer', 'cuenta de debito bbva']
['cuenta bancomer

In [8]:
#dataf5 = data4.pivot_table(index=['fh_corte'], columns='banco',values='tendencia').reset_index().rename_axis(None, axis=1)

In [9]:
#dataf5.to_excel('/Users/mb66483/Documents/exportprest.xlsx')